In [1]:
import  requests 
import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib3.exceptions import SSLError

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout

In [2]:
def getArticle(link):
    
    try: 
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = link
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")
#         data = soup.findAll('div',{'class':'article_top_slogan'})
#         slogan = data[0].text
        data = soup.findAll('div',{'class':'article_body'})
        article = data[0].text
        
        return article

    except HTTPError as http_err:
#         print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
#         print(f'Other1 error occurred: {err}')  
        pass

In [3]:
search_input = 'Turkish Airlines'
search_string = '+'.join(str(e) for e in search_input.lower().split())

## each search page has number of results pages with a limit of 50 news article in each one.

url1 = f'https://www.dailysabah.com/search?query={search_string}&qlimit=by_fifty&pgno='
url2 = f'https://www.dailysabah.com/search?query={search_string}&qlimit=by_fifty&qsort=oldest&pgno='
urls = [url1,url2]
print(url1)
print(url2)


https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&pgno=
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&qsort=oldest&pgno=


In [4]:
list_of_links = []

for url in urls:
    numbers = 1
    try: 
    
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = url
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")
    
        data = soup.findAll('div',{'class':'pages_list'})
        data2 = str(data).split('pgno=')
        number = int(data2[-2].split('">')[0])
        
        for i in range(1,number+1,1):
            list_of_links.append(url+str(i))
            
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
        print(f'Other1 error occurred: {err}')  
        pass

    
print(len(list_of_links))
print(list_of_links[0])
print(list_of_links[-1])

102
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&pgno=1
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&qsort=oldest&pgno=51


In [5]:
headlines = []
for url in list_of_links:
    try: 
    
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = url
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")
    
        data = soup.findAll('ul',{'class':'items_list'})
        data2 = str(data[0]).split('<div class="image_holder">')

        for i in range(1,len(data2),2):
            link = data2[i].split('<a href="')[1].split('">')[0]
            date =  data2[i].split('<div class="date_text">')[1].split('</div')[0].strip()
            headline = data2[i].split('<h3>')[1].split('</a>')[0].split('">')[1].strip()
#             print(link)
            article = getArticle(link)
            
            headlines.append([headline,date,article,link])
            
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
        print(f'Other1 error occurred: {err}')  
        pass


In [6]:
print(headlines[-3])

print(len(headlines))

["Turkey's exports expected to bounce back this month", 'Sep 03, 2015', "\n                    The slowdown in the rate of the fall of Turkey's exports in August has corroborated Turkish Exporters Assembly (TİM) President Mehmet Büyükekşi's argument that exports will start to rise in September. Exports dropped from 13 percent in July to 5 percent in August.\n\r\nTurkey's exports decreased to $10.5 billion with a 4.9 percent fall in August when compared to the same month last year. This decline was around 13 percent in July. Turkey's exports dropped 8.9 percent to $95.135 billion between January 2015 and August 2015.\n\r\nBüyükekşi announced export figures for this period and said the decline in exports stemmed from negative developments in foreign markets and the fall in euro-dollar parity. The fall in the parity caused a loss of $8.9 billion in the first eight months of the year, according to Büyükekşi.\n\r\nWarning companies against speculative exchange rate movements, Büyükekşi said

In [8]:
    
df = pd.DataFrame(headlines, columns =['text','date','article','article_link'])
df.to_csv(f'apis_datasets/{search_input}_DailySabah.csv', encoding='utf-8', index=False)

# df.head()  
# df.describe

In [11]:
df.head()


,text,date,article,article_link
0,Turkish airports host 9.5M passengers in Augus...,"Sep 10, 2020",\nThe Turkish civil aviation sector has surviv...,https://www.dailysabah.com/business/transporta...
1,Performance tests to start on high-speed metro...,"Sep 07, 2020",\nThe production of the first subway car of th...,https://www.dailysabah.com/business/transporta...
2,Ukrainian woman returning from Antalya takes w...,"Sep 04, 2020",\nA woman arriving back from a vacation in Tur...,https://www.dailysabah.com/world/europe/ukrain...
3,Turkey-Iraq flights suspended until Oct. 1,"Sep 01, 2020",\nFlights between Turkey and Iraq have been su...,https://www.dailysabah.com/business/transporta...
4,Companies traded on Borsa Istanbul post $4.7B ...,"Aug 26, 2020",\nThe net profits of companies traded on the B...,https://www.dailysabah.com/business/finance/co...
